In [2]:
import numpy as np
import random
import cvxpy as cp
import matplotlib.pyplot as plt

(CVXPY) Apr 04 01:28:17 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Apr 04 01:28:17 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


The following is a summary of a research paper:
 
Beveridge, Andrew, and Stan Wagon. “The Sorting Hat Goes to college.” Mathematics Magazine, vol. 87, no. 4, Oct. 2014, pp. 243–251, https://doi.org/10.4169/math.mag.87.4.243. 

## Problem 

Assign first-years students to their desired courses (about 500 students must be placed into about 35 first-year courses). Maximize overall student satisfaction such that each student is assigned into a course that they ranked. 

## Solution

### Hungarian Algorithm (minimum weight perfect matching algorithm)

Weighted bipartite graph - n vertices in one part correspond to n vertices in the other part

- create a bipartite graph that enforced the constraints on class size and demographics
- choose a weighting scheme that ensured trade-offs were consistent with the goals an priorities of the AP office

The Hungarian algorithm works well but sometimes the constraints cannot be met, which is why the Integer Linear Programming method works better as it handles a wide variety of constraints.


### Integer Programming problem

**Parameters:**
- students $S_i, 1 \leq i \leq n$
- courses $C_j, 1 \leq j \leq m$
- M, F, I are indices corresponding to male, female, and international students
- preferences $P_i$
- $X_j$ : set of students i having $C_j$ as one of their choices, where $j \in P_i$
 
\* worst case $X_j = \emptyset$

**Decision Variables**
- $x_{i,j}$ = 1 if for each pair $(S_i, C_j)$ where $j \in P_i$, $S_i$ is assigned to $C_j$, and 0 otherwise
- $\sum_{j \in P_i} x_{i,j}$ = 1 for each $i \leq n$ and $x_{i,j} \in \{0,1\}$

**Objective Function**
$$\text{min } \sum_{i=1}^n x_{i,j_{i,1}} + \alpha x_{i,j_{i,2}} + \alpha^2 x_{i,j_{i,3}}  + \alpha^3 x_{i,j_{i,4}}$$ 
where $j_{i,1}, ..., j_{i,4}$ are student $S_i$'s four choices and $\alpha > 1$ 

* $\alpha$ is a value chosen according to the trade-offs for swapping students between classes
The weight of placing a student in a course increases as it is further from the first choice.

**Constraints**
1. Each class size should be from 10 to 16 students
2. The demographics of each class should be roughly comparable to the entire student body (60% female and 11% international student)
* sometimes the problem gives infeasible solutions, so the students are placed into an unranked course

**Course size constraint**

The sum of the number of students enrolled in each course $C_j$ ranges from L to U:
- $L \leq \sum_{i \in X_j} x_{i,j} \leq U$ for each $j \leq m$

Typically, we have $(L, U) = (12, 16)$ so the course can have 12 to 16 students. However, it is common to have a course that allows more than 16 or less than 12. We introduce $q_{j,s}$ where s is the size of $C_j$ and $9 \leq s \leq 17$. Suppose we have $Q_{17}, Q_{11}, Q_{10}, Q_9$ that represents the maximum number of students allowed in each class 17, 11, 10, and 9 respectively. Then, the ideal size of each $C_j$ for $Q_{17}, Q_{11}, Q_{10}, Q_9$ are:
* $\sum_j q_{j,17} \leq Q_{17}$ 
* $\sum_j q_{j,11} \leq Q_{11}$ 
* $\sum_j q_{j,10} \leq Q_{10}$ 
* $\sum_j q_{j,9} \leq Q_{9}$ 

However, we need to make sure that each course only takes a unique maximum number of students ranging from 9 to 17:
$$\sum_{s=9}^{17} q_{j,s} = 1 \quad \forall j$$

Therefore, we have the total number of students enrolled in $C_j$ be equal to the maximum number of students a course can have by multiplying s with $\sum_{s=9}^{17} q_{j,s}$
$$\sum_{i \in X_j} x_{i,j} = \sum_{s=9}^{17} sq_{j,s} \quad \forall j$$ 


**Gender constraint**

There are at least 4 male and 4 female in each course

* $\sum_{i \in M \cap X_j} x_{i,j} \geq 4$
* $\sum_{i \in F \cap X_j} x_{i,j} \geq 4$

The same idea from the course size constraint can be used for the gender constraint to help loosened the constraint of having at least 4 male and female.

The paper introduced a new variable $y_{m,j} = 1$ when $C_j$ has a value of m males, and 0 otherwise. So, we can make sure that $C_j$ has a maximum of m number of males:
$$\sum_{m=0}^{17} y_{m,j} = 1 \quad \forall j$$
and thus, the total number of male students enrolled in $C_j$ can be equal to the maximum number of male students a course can have by multiplying m with $\sum_{m=0}^{17} y_{m,j}$:
$$\sum_{i \in M \cap X_j} x_{i,j} = \sum_{m=0}^{17} my_{m,j} \quad \forall j$$

Similarly, we will have the same formula for the number of females in a course:
$$\sum_{i \in F \cap X_j} x_{i,j} = \sum_{f=0}^{17} fy_{f,j} \quad \forall j$$

**International constraint**

Each course contains at most B international students

$\sum_{i \in I \cap X_j} x_{i,j} \leq B \quad \forall j$





## Building Solution

In [28]:
### try obvious dataset
# [0,1,1,0,1,2,3] means student id 0, female, international student with preference of course 0,1,2,3 from first to last choice
s = np.array([[0, 1, 1, 0, 1, 2, 3],
               [1, 1, 0, 1, 2, 2, 3],
               [2, 0, 1, 0, 1, 2, 3],
               [3, 1, 1, 0, 1, 2, 3]])

P = s[:,-4:] # preference P of student i

# [0,5] means course id 0 with maximum 5 students in the class
c = np.array([[0,5],[1,4],[2,3],[3,2]])

a = 2
n = len(s)
m = len(c)
a = 2
x = cp.Variable((n, m), integer = True) # set a variable that can only have integer values

# temporary constraints to check objectives
constraints = [   
    cp.sum(x, axis=1) == 1,  # Each student must be assigned to exactly one course
    cp.sum(x, axis=0) <= c[:, 1],  # Each course can have at most its maximum number of students
    x >= 0
]

# objective function
# P[:,0] is getting the first column which is getting the first-ranked choice of each student
objective = cp.Minimize(cp.sum(x[:, P[:,0]] + a*x[:, P[:,1]] + a**2*x[:, P[:,2]] + a**3*x[:, P[:,3]]))
prob = cp.Problem(objective, constraints)
prob.solve(verbose=True)
x.value

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Apr 04 01:54:01 AM: Your problem has 16 variables, 3 constraints, and 0 parameters.
(CVXPY) Apr 04 01:54:01 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 04 01:54:01 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 04 01:54:01 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 04 01:54:01 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 04 01:54:01 AM: Compiling problem (target solver=SCIPY).
(CV

array([[ 1., -0., -0., -0.],
       [ 1., -0., -0., -0.],
       [ 1., -0., -0., -0.],
       [ 1., -0., -0., -0.]])

The above solution assigns each student to course id 0. 

### Defining the dataset

we have to create a dummy set for 544 students and 35 courses. The students will need to be classfied into male, female and international students. We need a separate array to define prefere.nces of the students by creating an ordered list of courses. Each courses also need the limitation for the number of students. 

In [39]:
n = 544 # number of students
m = 35 # number of courses

**Create courses dataset**

The course dataset will have 35 courses each with an id from 1 to 35 and a course limit ranging from 9 to 17. 

In [47]:
# create an array to store course number and 
courses = {}

for i in range(1,m+1): # iterate to fill m number of courses
    course_id = i
    # set random limit from 9 to 17 with more max_student = 16
    prob = [0.02, 0.02, 0.02, 0.03, 0.03, 0.05, 0.13, 0.42, 0.28]
    max_students = np.random.choice(range(9,18), p = prob) 
    # let i be the course id from 1 to 35
    courses[i] = {'max': max_students}

print(len(courses))


for i in list(courses.items())[:5]:
    print(i)

35
(1, {'max': 16})
(2, {'max': 16})
(3, {'max': 15})
(4, {'max': 12})
(5, {'max': 17})


The total number of seats should be more or equal to the total number of students.

In [52]:
# check how many empty seats
total_seats = 0
for course in courses.values():
    total_seats += course['max']

print(total_seats)

544


**Create student dataset**

The student dataset should not be completely random because according to the paper, the Machester college has 60% female students and 11% international students.

- Let gender be 1 for female and 0 otherwise. 
- Let international be 1 for international students and 0 otherwise.
- Let preferences be an ordered list of courses

In [42]:
# create a dictionary to store data
course_id = list(courses.keys())
students = {}
for i in range(1,n+1): # iterate to fill n students
    rand_gender = random.random()
    rand_international = random.random()
    
    # probability of having female student is 60%
    gender = 1 if (rand_gender > 0.6) else 0
    # probability of international student is 11%
    international = 1 if(rand_international < 0.11) else 0

    # create list of preference
    preference = random.sample(course_id, 4)
    # let i be the student id from 1 to 544
    students[i] = {'gender': gender, 'international': international,
                   'preference': preference}

In [43]:
for i in list(students.items())[:5]:
    print(i)

(1, {'gender': 0, 'international': 0, 'preference': [4, 15, 30, 29]})
(2, {'gender': 0, 'international': 0, 'preference': [1, 2, 12, 23]})
(3, {'gender': 0, 'international': 1, 'preference': [29, 28, 18, 7]})
(4, {'gender': 1, 'international': 0, 'preference': [23, 25, 8, 24]})
(5, {'gender': 1, 'international': 0, 'preference': [29, 18, 1, 31]})


**Set the objective function**

$$\text{min } \sum_{i=1}^n x_{i,j_{i,1}} + \alpha x_{i,j_{i,2}} + \alpha^2 x_{i,j_{i,3}}  + \alpha^3 x_{i,j_{i,4}}$$ 
where $j_{i,1}, ..., j_{i,4}$ are student $S_i$'s four choices and $\alpha > 1$ 

In [50]:
# let alpha = 2
a = 2

# set decision variable
x = cp.Variable((n, m), integer = True)
# preference list P of student i 
P = np.array([students[i]['preference'] for i in range(1, n+1)])

# minimize the sum of the weighted choices for each student
objective = cp.Minimize(cp.sum(x[:, P[:,0]-1] + a*x[:, P[:,1]-1] + a**2*x[:, P[:,2]-1] + a**3*x[:, P[:,3]-1]))

constraints = [   
    cp.sum(x, axis=1) == 1,  # Each student must be assigned to exactly one course
    cp.sum(x, axis=0) <= [courses[i]['max'] for i in range(1, m+1)],  # Each course can have at most its maximum number of students
    x >= 0
]

prob = cp.Problem(objective, constraints)
prob.solve()
x.value

array([[-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       ...,
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.]])

In [53]:
check = True
# in each row the sum should be equal to one since students are assigned to only one class
for n in x.value:
    if sum(n) == 1: check = True
    else: False

print(check)

True


Since it's true, then the objective function is set properly.

**Set course constraints**
$$\sum_{s=9}^{17} q_{j,s} = 1$$
$$\sum_{i \in X_j} x_{i,j} = \sum_{s=9}^{17} sq_{j,s}$$ 

In [234]:
course_constraint = []

# create variable q and let it have a size of m by the range of course size (18-9) 
size = 9
q = cp.Variable((m, size), integer = True)
# constraint for checking each course only have one maximum size
course_constraint += [cp.sum(q[j,:]) == 1 for j in range(m)]
# sum of student i for each class j should be equal to the maximum number of the class
course_constraint += [cp.sum(x[:, j]) == cp.sum([s * q[j, s] for s in range(9)]) for j in range(m)]

**Set gender constraints**

$$\sum_{m=0}^{17} y_{m,j} = 1$$
$$\sum_{f=0}^{17} y_{f,j} = 1$$
$$\sum_{i \in M \cap X_j} x_{i,j} = \sum_{m=0}^{17} my_{m,j}$$
$$\sum_{i \in F \cap X_j} x_{i,j} = \sum_{f=0}^{17} fy_{f,j}$$

In [235]:
gender_constraint = []
# create new variable for y_{m,j}
ym = cp.Variable((17, m), integer = True)
# for each course j, there should be only 1 maximum m number of males
gender_constraint += [cp.sum(ym[:,j]) == 1 for j in range(m)]
# create new variable for y_{f,j}
yf = cp.Variable((17, m), integer = True)
# for each course j, there should be only 1 maximum f number of females
gender_constraint += [cp.sum(yf[:,j]) == 1 for j in range(m)]

# for each course j, the number of males also in preference of student i will be equal to the maximum number of males
male_constraint = [cp.sum([x[i, j-1] for i in range(n) if students[i+1]['gender'] == 0
                                    and j in students[i+1]['preference']]) 
                                    == cp.sum([m * ym[m, j-1] for m in range(9)])
                                    for j in range(1, m+1)]

# for each course j, the number of females also in preference of student i will be equal to the maximum number of females
female_constraint = [cp.sum([x[i, j-1] for i in range(n) if students[i+1]['gender'] == 1
                                    and j in students[i+1]['preference']]) 
                                    == cp.sum([m * yf[m, j-1] for m in range(9)])
                                    for j in range(1, m+1)]

gender_constraint += male_constraint + female_constraint

**Set international students constraints**

$$\sum_{i \in I \cap X_j} x_{i,j} \leq 8$$

In [236]:
international_constraint = [cp.sum([x[i, j-1] for i in range(n) if students[i+1]['international'] == 1
                                    and j in students[i+1]['preference']]) <= 8 for j in range(1, m+1)]

**Solve**

In [237]:
constraints = course_constraint + gender_constraint + international_constraint
problem = cp.Problem(objective, constraints)
problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Apr 02 11:29:27 PM: Your problem has 20545 variables, 245 constraints, and 0 parameters.


(CVXPY) Apr 02 11:29:27 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 02 11:29:27 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 02 11:29:27 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 02 11:29:27 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 02 11:29:28 PM: Compiling problem (target solver=SCIPY).
(CVXPY) Apr 02 11:29:28 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCIPY
(CVXPY) Apr 02 11:29:28 PM: Applying reduction Dcp2Cone
(CVXPY) Apr 02 11:29:28 PM: Applying reduction CvxAttr2Constr
(CVXPY) Apr 02 11:29:28 PM: Applyi

SolverError: Solver 'SCIPY' failed. Try another solver, or solve with verbose=True for more information.